# ML Reproducibility Challenge 2022
## TS2Vec: Towards Universal Representation of Time Series

The original paper can be found [here](https://arxiv.org/pdf/2106.10466.pdf)

Original code implementation can be found on [github/](https://github.com/yuezhihan/ts2vec.git)

ML Reproducibility Challenge 2022 [homepage](https://paperswithcode.com/rc2022)

In [ ]:
!git clone https://github.com/yuezhihan/ts2vec.git # clones TS2Vec repo
!python3 --version # want: 3.8.15

## Preparing the Colab Environment

Our Google Colab environment came with a python3.7 kernel installed. TS2Vec requires python3.8, so we had to preparing the environment accordingly. 

Make sure the runtime type is set to 'gpu'.

Installs python3.8 and library requirements. 

In [ ]:
!sudo apt install python3.8
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 2
!python3 --version # want: 3.8.15

In [ ]:
# Updating python version breaks pip -- get pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py

# install requirements - their requirements.txt file didn't work
!pip3 install torch
!pip3 install numpy
!pip3 install scikit_learn==0.24.2
!pip3 install bottleneck
!pip3 install pandas
!pip3 install statsmodels
!pip3 install tables

In [ ]:
%cd ts2vec # change directory to cloned repo

## Reproducing paper results

Performance metrics presented in the results section can be reproduced by running TS2Vec packaged functions provided in the $\texttt{scripts/}$ folder.

Datasets must be manually prepared according to instructions in the TS2Vec $\texttt{README.md}$.

In [ ]:
# sample reproduction line
!python3 -u train.py ETTh1 forecast_univar --loader forecast_csv_univar --repr-dims 320 --max-threads 8 --seed 42 --eval

## Running TS2Vec on a new dataset

The dataset is of normalized hydrologic data (groundwater depth and local precipitation) measured over the course of 3 months (June-September 2021). 

If using Google Colab, you'll need to upload the data to the $\texttt{datasets/}$ folder before running the following cells. 

In [ ]:
# run TS2Vec forecasting on hydrologic dataset with all default parameters
!python3 train.py norm_002B_rain newdata_run --loader forecast_csv --eval

## Visualizing Forecasting Performance

To visualize the forecast predicitons, we had to modify several functions from the $\texttt{tasks/}$ folder.

In [ ]:
# import functions from TS2Vec 
from ts2vec import TS2Vec
from tasks.forecasting import eval_forecasting, generate_pred_samples, cal_metrics

# import libraries
import datautils
import numpy as np
import time
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

In [ ]:
'''
We modified fit_ridge(), the function that fits a Ridge Regression optimizer for the forecasting task
in order to allow a choice of a wider range of alpha values (parameter to Ridge()). 
'''
def fit_ridge(train_features, train_y, valid_features, valid_y, MAX_SAMPLES=100000):
    # If the training set is too large, subsample MAX_SAMPLES examples
    if train_features.shape[0] > MAX_SAMPLES:
        split = train_test_split(
            train_features, train_y,
            train_size=MAX_SAMPLES, random_state=0
        )
        train_features = split[0]
        train_y = split[2]
    if valid_features.shape[0] > MAX_SAMPLES:
        split = train_test_split(
            valid_features, valid_y,
            train_size=MAX_SAMPLES, random_state=0
        )
        valid_features = split[0]
        valid_y = split[2]
    
    alphas = [0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 30, 40, 50, 75, 100, 150, 200, 250, 300, 350, 400, 450, 500, \
              550, 600, 650, 700, 750, 800, 850, 900, 950, 1000]
    valid_results = []
    # chooses the alpha with the best score
    for alpha in alphas:
        lr = Ridge(alpha=alpha).fit(train_features, train_y)
        valid_pred = lr.predict(valid_features)
        score = np.sqrt(((valid_pred - valid_y) ** 2).mean()) + np.abs(valid_pred - valid_y).mean()
        valid_results.append(score)
    best_alpha = alphas[np.argmin(valid_results)]
    print("this is the best alpha ever: ", best_alpha)
    
    lr = Ridge(alpha=best_alpha)
    lr.fit(train_features, train_y)
    return lr

In [ ]:
'''
Modified the eval_forecasting() function to return the actual test prediction and labels, which 
previously weren't accessible. 
'''
def mod_eval_forecasting(model, data, train_slice, valid_slice, test_slice, scaler, pred_lens, 
                         n_covariate_cols, sliding_length=1, padding=200,batch_size=256):
    
    t = time.time()
    all_repr = model.encode(
        data,
        casual=True,
        sliding_length=sliding_length,
        sliding_padding=padding,
        batch_size=batch_size 
    )
    ts2vec_infer_time = time.time() - t
    
    train_repr = all_repr[:, train_slice]
    valid_repr = all_repr[:, valid_slice]
    test_repr = all_repr[:, test_slice]
    
    train_data = data[:, train_slice, n_covariate_cols:]
    valid_data = data[:, valid_slice, n_covariate_cols:]
    test_data = data[:, test_slice, n_covariate_cols:]
    
    ours_result = {}
    lr_train_time = {}
    lr_infer_time = {}
    out_log = {}
    collect_test_pred = {}
    collect_test_labels = {}
    for pred_len in pred_lens:
        print("pred_len", pred_len)
        train_features, train_labels = generate_pred_samples(train_repr, train_data, pred_len, drop=padding)
        valid_features, valid_labels = generate_pred_samples(valid_repr, valid_data, pred_len)
        test_features, test_labels = generate_pred_samples(test_repr, test_data, pred_len)
        
        t = time.time()
        lr = fit_ridge(train_features, train_labels, valid_features, valid_labels)
        lr_train_time[pred_len] = time.time() - t
        
        t = time.time()
        test_pred = lr.predict(test_features)
        lr_infer_time[pred_len] = time.time() - t

        ori_shape = test_data.shape[0], -1, pred_len, test_data.shape[2]
        test_pred = test_pred.reshape(ori_shape)
        test_labels = test_labels.reshape(ori_shape)
        collect_test_pred[pred_len] = test_pred
        collect_test_labels[pred_len] = test_labels

    return collect_test_pred, collect_test_labels

## Training models

We ran a hyperparameter search, documented in the commented out lines below. Model files are not included here, but can reproduced by training a new model with the specified arguments and then modifying the file name.  

In [ ]:
!python train.py norm_002B_rain forecast_multivar --loader forecast_csv --repr-dims 120 --epochs 800 --eval

In [ ]:
# load the model
model_type = "multi"
 
if model_type == "multi" :
    in_dim = 10

 #### MODIFYING REPR-DIMS ###
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_183101/model.pkl'#repr-dims=50
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_182755/model.pkl'#repr-dims=100
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_182047/model.pkl'#repr-dims=120
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_183334/model.pkl'#repr-dims=160
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_183549/model.pkl'#repr-dims=320
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_183849/model.pkl'#repr-dims=640
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_184141/model.pkl'#repr-dims=1280

 ### MODIFYING EPOCHS WITH REPR-DIM=120 ###
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_184705/model.pkl'#repr-dims=100, epochs=400
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_184952/model.pkl'#repr-dims=120, epochs=600
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_185230/model.pkl'#repr-dims=120, epochs=800
 
 ### MODIFYING LR WITH REPR-DIM=120 EPOCHS=400 ###
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_185750/model.pkl'#repr-dims=120, epochs=400, lr=0.0001
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_190131/model.pkl'#repr-dims=120, epochs=800, lr=0.01
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_190401/model.pkl'#repr-dims=120, epochs=400, lr=0.1

 ### MODIFYING MAX TRAIN LENGTH REPR-DIM=120 EPOCHS=400 LR=(DEFAULT) ###
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_190812/model.pkl'#repr-dims=120, epochs=400, mtl=500
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_191052/model.pkl'#repr-dims=120, epochs=400, mtl=1000
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_191326/model.pkl'#repr-dims=120, epochs=400, mtl=4000
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_191603/model.pkl'#repr-dims=120, epochs=400, mtl=5000
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_191839/model.pkl'#repr-dims=120, epochs=400, mtl=6000
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_192112/model.pkl'#repr-dims=120, epochs=400, mtl=8000
 
 ### MODIFYING alpha in the losses.py hierarchical contrastive loss 
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_192832/model.pkl' #--repr-dims 120 --epochs 800, alpha = 0.9
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_193237/model.pkl' #--repr-dims 120 --epochs 800, alpha = 0.99
#  file_path = 'training/norm_002B_rain__forecast_multivar_20221209_193556/model.pkl' #--repr-dims 120 --epochs 800, alpha = 0.1
file_path = 'training/norm_002B_rain__forecast_multivar_20221209_193912/model.pkl' #--repr-dims 120 --epochs 800, alpha = 0.01

rep = 120
is_univar = False
model = TS2Vec(input_dims=in_dim, output_dims=rep)

elif model_type == "uni" :
    in_dim = 8
    file_path = 'training/hey_002B_rain__forecast_multivar_20221201_203016/model.pkl'
    is_univar = True
    model = TS2Vec(input_dims=in_dim)

model.load(file_path)

# prepare the datasets
data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols = datautils.load_forecast_csv('norm_002B_rain', univar=is_univar)

### Visualize the data input to the model

In [ ]:
print(np.shape(data))

In [ ]:
# Plot time-encoded input data
fig, ax = plt.subplots(figsize=(12,6),nrows=2, sharex=True)
x = np.linspace(0,data.shape[1]-1, data.shape[1])
ax[0].scatter(x, data[0,:,-1], marker='.')
ax[0].set_title("Original Groundwater Timeseries")
for i in range(in_dim):
    ax[1].scatter(x, data[0,:,i], marker='.', alpha=0.5, label=i)
ax[1].set_title("Expanded dimensional data")
ax[1].legend()
plt.tight_layout()

### Fit the model to the data

In [ ]:
sl = 1
p=200
bs = 256
test_pred, test_labels = mod_eval_forecasting(model, data, train_slice, valid_slice, test_slice, scaler, pred_lens, 
                                              n_covariate_cols, sliding_length=sl, padding=p, batch_size=bs)

### Plot the best slices

In [ ]:
# the value in test_pred[] and test_labels[] is the H in pred length
# selcet final dimension
dim_slice = -1

# plot prediction forecast for all pred_lengths
fig, ax = plt.subplots(figsize=(12,8),nrows=len(pred_lens), sharex=True, sharey=False)

for iter, h in enumerate(pred_lens):
    preds = test_pred[h]
    labs = test_labels[h]
    # use the slice that has the lowest difference between lables and predictions
    diffs = np.sum(np.abs(labs-preds), axis = (0,2))
    # get the best slice index for each n_covariant_col
    best_pred_inds = np.argmin(diffs, axis = 0)
    best_pred_ind = best_pred_inds[-1]
    # time steps
    t = np.arange(h) 

    ax[iter].plot(t, labs[0,best_pred_ind,:,dim_slice], label = 'Ground Truth', linewidth=6)
    ax[iter].plot(t,  preds[0,best_pred_ind,:,dim_slice], marker='.', c='orange', label='TS2Vec')
    ax[iter].set_title('Prediction, pred_length=%i, ' %h + 'slice=%i'%best_pred_ind)
ax[-1].set_xlabel('Forecasting step')
ax[0].legend()
plt.tight_layout()

### Plot an interesting slice

In [ ]:
# the value in test_pred[] and test_labels[] is the H in pred length
# selcet final dimension
dim_slice = -1

# plot prediction forecast for all pred_lengths
fig, ax = plt.subplots(figsize=(12,8),nrows=len(pred_lens), sharex=True, sharey=False)

for iter, h in enumerate(pred_lens):
    preds = test_pred[h]
    labs = test_labels[h]
    best_pred_ind = 1300
    # time steps
    t = np.arange(h) 

    ax[iter].plot(t, labs[0,best_pred_ind,:,dim_slice], label = 'Ground Truth', linewidth=6)
    ax[iter].plot(t,  preds[0,best_pred_ind,:,dim_slice], marker='.', c='orange', label='TS2Vec')
    ax[iter].set_title('Prediction, pred_length=%i, ' %h + 'slice=%i'%best_pred_ind)
ax[-1].set_xlabel('Forecasting step')
ax[0].legend()
plt.tight_layout()